In [1]:
import h3, folium, requests, json
# !pip install git+https://github.com/sabman/h3-utils.git@main#egg=h3_utils
from h3_utils import tools as h3_tools
from h3_utils import viz as h3_viz

In [2]:
# if file is in data load that
geojson_file = "../tests/data/councils/holdfastbay/jettyroad.geojson"

with open(geojson_file) as f:
    data = json.load(f)

if not data:
  # get the data from the server (could be a request to geodb-cartodb)
  url = "https://gist.githubusercontent.com/sabman/f1aee8de222c263fb0aa4d40409404e7/raw/118150c1d402d5a998a5b180731a98ac33983dad/jettyroad.geojson"
  r = requests.get(url)
  data = r.json()

In [3]:
# We only need the geometry fro the next steps
geoJson = data['features'][0]['geometry']

In [4]:
h3_viz.create_hexagons_map(geoJson, resolution=6)

# How to use `find_resolution_for_geojson`

This function is used to find an h3 resolution that matches the requirements for coverage of the input geojson by the ratio of the area covering the geojson to the total area of the cells. The function will return the lowest resolution that meets the requirements.

See example usage below:

In [5]:
resolution, cells = h3_tools.find_resolution_for_geojson(geoJson, 0.85, 1)

In [6]:
h3_viz.create_map_with_hexagons(geoJson, cells)